Resume NER Part 4: Working with Flair NLP

---

In this part we will use flair NLP to train a model on our data and evaluate the results. Please make sure you have set up your Google account and uploaded your files to Google drive. This Notebook should run on Google Colab.

Let's change the working directory to the Google drive where our training data is, and install flair nlp. 

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [10]:
import os
os.chdir("/content/gdrive/My Drive/saki_ex2") 
print(os.getcwd())

/content/gdrive/My Drive/saki_ex2


In [11]:
# download flair library #
! pip install flair

In the next section, we will train a NER model with flair. This code is taken from the flair nlp tutorials section 7. "Training a model" 
https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_7_TRAINING_A_MODEL.md



In [0]:
# imports 
from flair.data import Corpus
from flair.data_fetcher import NLPTaskDataFetcher, NLPTask
from typing import List

# columns of "gold standard" ner annotations and text "text","ner","doc","ner_spacy" doc ner ner_spacy text
columns = {0: "doc", 1: "ner", 2: "ner_spacy", 3: "text"}
#columns = {0: "text", 1: "ner", 2: "ner_spacy", 3: "doc"}
# folder where training and test data are
data_folder = '/content/gdrive/My Drive/saki_ex2/data/'
train_file = 'train3_res_bilou_preprocessed.txt'
test_file = 'test3_res_bilou_preprocessed.txt'

# 2. what tag do we want to predict?
tag_type = 'ner'


In [13]:
downsample = 1.0 # 1.0 is full data, try a much smaller number like 0.01 to test run the code train_res_bilou_preprocessed.txt test_res_bilou_preprocessed.txt
# 1. get the corpus
corpus: Corpus = NLPTaskDataFetcher.load_column_corpus(data_folder, columns, train_file=train_file, test_file=test_file, dev_file=None).downsample(downsample)
print(corpus)

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary.idx2item)


2019-06-18 22:46:21,563 Reading data from /content/gdrive/My Drive/saki_ex2/data
2019-06-18 22:46:21,565 Train: /content/gdrive/My Drive/saki_ex2/data/train3_res_bilou_preprocessed.txt
2019-06-18 22:46:21,566 Dev: None
2019-06-18 22:46:21,567 Test: /content/gdrive/My Drive/saki_ex2/data/test3_res_bilou_preprocessed.txt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated function (or staticmethod) load_column_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:312: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  train_file, column_format
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:318: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  test_file, column_format


Corpus: 9665 train + 1074 dev + 3681 test sentences
[b'<unk>', b'O', b'B-Degree', b'L-Degree', b'I-Degree', b'"B-College', b'"I-College', b'"L-College', b'B-Designation', b'L-Designation', b'U-Degree', b'I-Designation', b'-', b'U-Designation', b'"U-College', b'ner', b'<START>', b'<STOP>']


In [0]:

# 4. initialize embeddings. Experiment with different embedding types to see what gets the best results
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings,FlairEmbeddings, CharacterEmbeddings
from typing import List
embedding_types: List[TokenEmbeddings] = [
    #WordEmbeddings('glove'),
    # comment in this line to use character embeddings
    CharacterEmbeddings(),

    # comment in these lines to use flair embeddings (needs a LONG time to train :-)
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)




In [15]:
# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/resume-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=40)

# 8. plot training curves (optional)
#from flair.visual.training_curves import Plotter
#plotter = Plotter()
#plotter.plot_training_curves('resources/taggers/example-ner/loss.tsv')
#plotter.plot_weights('resources/taggers/example-ner/weights.txt')


2019-06-18 22:46:44,700 ----------------------------------------------------------------------------------------------------
2019-06-18 22:46:44,702 Evaluation method: MICRO_F1_SCORE
2019-06-18 22:46:45,155 ----------------------------------------------------------------------------------------------------
2019-06-18 22:46:46,660 epoch 1 - iter 0/303 - loss 55.26952362
2019-06-18 22:47:22,522 epoch 1 - iter 30/303 - loss 8.07839002
2019-06-18 22:47:57,325 epoch 1 - iter 60/303 - loss 5.74743993
2019-06-18 22:48:31,944 epoch 1 - iter 90/303 - loss 4.95939645
2019-06-18 22:49:06,788 epoch 1 - iter 120/303 - loss 4.37652885
2019-06-18 22:49:54,126 epoch 1 - iter 150/303 - loss 3.98311502
2019-06-18 22:50:34,519 epoch 1 - iter 180/303 - loss 3.67259192
2019-06-18 22:51:11,693 epoch 1 - iter 210/303 - loss 3.41577864
2019-06-18 22:51:51,461 epoch 1 - iter 240/303 - loss 3.19970657
2019-06-18 22:52:29,199 epoch 1 - iter 270/303 - loss 3.05496541
2019-06-18 22:53:08,650 epoch 1 - iter 300/303

{'dev_loss_history': [tensor(1.3839, device='cuda:0'),
  tensor(1.0108, device='cuda:0'),
  tensor(0.8811, device='cuda:0'),
  tensor(0.7503, device='cuda:0'),
  tensor(0.6701, device='cuda:0'),
  tensor(0.6605, device='cuda:0'),
  tensor(0.6195, device='cuda:0'),
  tensor(0.5710, device='cuda:0'),
  tensor(0.7196, device='cuda:0'),
  tensor(0.5236, device='cuda:0'),
  tensor(0.5119, device='cuda:0'),
  tensor(0.5358, device='cuda:0'),
  tensor(0.5148, device='cuda:0'),
  tensor(0.5012, device='cuda:0'),
  tensor(0.5030, device='cuda:0'),
  tensor(0.4685, device='cuda:0'),
  tensor(0.4826, device='cuda:0'),
  tensor(0.4683, device='cuda:0'),
  tensor(0.4937, device='cuda:0'),
  tensor(0.4811, device='cuda:0'),
  tensor(0.4498, device='cuda:0'),
  tensor(0.4434, device='cuda:0'),
  tensor(0.4387, device='cuda:0'),
  tensor(0.4526, device='cuda:0'),
  tensor(0.4411, device='cuda:0'),
  tensor(0.4477, device='cuda:0'),
  tensor(0.4494, device='cuda:0'),
  tensor(0.4475, device='cuda:0'),
